In [1]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')
debug = False

Mounted at /content/drive


In [ ]:
model_path = "/content/drive/MyDrive/thesis/2_MODELING/base-best_3/checkpoint-76104"

In [5]:
from sentence_transformers import (SentenceTransformer, util)
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
model = SentenceTransformer(model_path,  device=device, )


Using device: cuda


Experimento cualitativo 1

In [51]:
vacante = [
    """En STF GROUP, compuesto por las marcas Studio F, Ela y STF MAN,
    creemos en el poder transformador de la juventud en el ámbito de las ventas.
    Buscamos personas con energía, deseos de aprender y una actitud positiva para
    integrarse a nuestro equipo. Como Asesor de Moda Temporada en Cali, tendrás la
    oportunidad de experimentar el mundo de la moda sin necesidad de experiencia previa,
    ya que valoramos tu actitud y pasión. Ofrecemos vacantes temporales y fijas, ideales
    para quienes desean estabilidad y crecimiento en la compañía de moda más grande de Colombia.


Responsabilidades:

    Atender y asesorar a los clientes en la tienda.
    Mantener la organización y presentación de productos.
    Cumplir con los objetivos de ventas establecidos.
    Contribuir a una experiencia de compra positiva para los clientes.

Requerimientos:

    Ser mayor de edad.
    Para vacantes fijas: experiencia en ventas de al menos 3 meses.
    Actitud positiva y deseos de crecer profesionalmente.


"""
]

vacant_embs = model.encode(vacante, convert_to_tensor=True, batch_size=32)
print(vacant_embs.shape)   # e.g., torch.Size([2, 768])

torch.Size([1, 768])


In [53]:
perfiles_experimento_1 = [
    # PERFIL A – ideal (debería salir 1º)
    """Persona joven, mayor de edad, con 1 año de experiencia continua como asesor(a) de moda
    en tienda de ropa femenina. Responsable de atención al cliente, sugerencia de outfits,
    manejo de probadores, apoyo en inventarios y cumplimiento de metas diarias de ventas.
    Alta energía, gusto por la moda y disponibilidad para trabajar fines de semana.""",

    # PERFIL B – experiencia en ventas, pero no en moda (debería salir medio)
    """Persona mayor de edad, con 1 año de experiencia como cajero y vendedor en supermercado.
    Atención al cliente, manejo de caja, reposición de góndolas y apoyo en inventarios.
    Acostumbrado(a) a trabajar con metas básicas de ventas y en horarios rotativos,
    con interés reciente por aprender del sector moda.""",

    # PERFIL C – sin experiencia, pero con interés en moda (debería salir medio/bajo)
    """Persona joven, recién egresada de bachillerato, sin experiencia formal en ventas.
    Muy interesada en tendencias de moda, redes sociales de moda y asesoría de imagen entre amigos.
    Actitud muy positiva, disposición total para aprender y para trabajar en temporada alta.""",

    # PERFIL D – carpintero (baja afinidad)
    """Trabajador con experiencia en carpintería y manufactura de muebles.
    Manejo de herramientas, lectura de planos, instalación y montaje en obra.
    Poco contacto con clientes finales y sin experiencia en retail ni en moda.""",

    # PERFIL E – ejecutivo comercial B2B (ventas pero muy senior y otro contexto)
    """Profesional con 10 años de experiencia como ejecutivo comercial en servicios B2B.
    Manejo de cuentas corporativas, negociación de contratos, elaboración de propuestas.
    Acostumbrado a trabajar por grandes metas de facturación, pero sin experiencia en piso de venta
    ni en asesoría de moda en tienda."""
]


candidate_embs = model.encode(perfiles_experimento_1, convert_to_tensor=True, batch_size=32)
print(candidate_embs.shape)   # e.g., torch.Size([2, 768])

torch.Size([5, 768])


In [70]:
vacant_embs = model.encode(vacante, convert_to_tensor=True)
candidate_embs = model.encode(perfiles_experimento_1, convert_to_tensor=True)

nombres_cortos = [
    "Moda Retail (muy apto)",
    "Apto en ventas (medio apto)",
    "Apto en moda (medio apto)",
    "Carpintería (poco apto)",
    "Seguros Gerencial (poco apto)"
]

scores = util.dot_score(vacant_embs, candidate_embs)[0].cpu().tolist()

# Hacemos zip en el mismo orden para no desalinear
tuplas = list(zip(nombres_cortos, perfiles_experimento_1, scores))

# Ordenar por el score descendente (posición 2)
tuplas_ordenadas = sorted(tuplas, key=lambda x: x[2], reverse=True)

# Imprimir resultados
for nombre_corto, texto_completo, score in tuplas_ordenadas:
    print(f"{nombre_corto}: {score:.2%}")



Apto en moda (medio apto): 55.48%
Moda Retail (muy apto): 47.69%
Apto en ventas (medio apto): 33.37%
Carpintería (poco apto): 33.33%
Seguros Gerencial (poco apto): 12.08%


In [55]:
vacante_carpinteria = ["""
Empresa de carpintería y fabricación de mobiliario a medida busca Ayudante de Carpintería.
La persona apoyará en corte, lijado, armado y montaje de piezas en madera, siguiendo planos
y especificaciones técnicas. Se valora experiencia previa en talleres u obras, manejo de
herramientas manuales y eléctricas, y compromiso con la seguridad industrial.
Trabajo 100% presencial, horario diurno.
"""]
vacante_seguros = ["""
Importante compañía del sector asegurador busca Gerente Comercial con amplia experiencia
liderando equipos de ventas, definición de estrategias comerciales y apertura de nuevos mercados.
Se requiere trayectoria comprobable en seguros de vida, generales u otros productos financieros,
así como experiencia en negociaciones con clientes corporativos y manejo de indicadores de
rentabilidad. Rol directivo, con alcance regional o nacional.
"""]


In [62]:
vacantes = [vacante, vacante_carpinteria, vacante_seguros]
nombres_vacantes = ["Moda Retail", "Carpintería", "Seguros Gerencial"]
perfiles_experimento_2 = [perfiles_experimento_1[0], perfiles_experimento_1[3], perfiles_experimento_1[4]]
perfiles = ["Moda Retail", "Carpintería", "Seguros Gerencial"]
for v_text, nombre_v in zip(vacantes, nombres_vacantes):
    v_emb = model.encode(v_text, convert_to_tensor=True)
    c_embs = model.encode(perfiles_experimento_2, convert_to_tensor=True)
    scores = util.dot_score(v_emb, c_embs)[0].cpu().tolist()
    print(f"\n=== Vacante: {nombre_v} ===")
    for i, (perfil, score) in enumerate(zip(perfiles_experimento_2, scores), start=1):
      print(f"Perfil {perfiles[i-1]} -> score: {score:.2%}")



=== Vacante: Moda Retail ===
Perfil Moda Retail -> score: 47.69%
Perfil Carpintería -> score: 33.33%
Perfil Seguros Gerencial -> score: 12.08%

=== Vacante: Carpintería ===
Perfil Moda Retail -> score: 17.36%
Perfil Carpintería -> score: 63.26%
Perfil Seguros Gerencial -> score: 15.71%

=== Vacante: Seguros Gerencial ===
Perfil Moda Retail -> score: 17.32%
Perfil Carpintería -> score: 24.66%
Perfil Seguros Gerencial -> score: 41.04%


In [ ]:

def correr_experimento(nombre_experimento, vacante_text_list, perfiles, nombres_perfiles):
    """
    vacante_text_list: lista con 1 string (la descripción de la vacante)
    perfiles: lista de strings (perfiles/cvs)
    nombres_perfiles: lista de etiquetas breves para cada perfil
    """
    print("\n" + "="*80)
    print(f"RESULTADOS {nombre_experimento}")
    print("="*80)

    # Embeddings
    vac_emb = model.encode(vacante_text_list, convert_to_tensor=True)
    perf_embs = model.encode(perfiles, convert_to_tensor=True)

    # Similaridad (dot score)
    scores = util.dot_score(vac_emb, perf_embs)[0].cpu().tolist()

    # Ordenar de mayor a menor score
    resultados = sorted(
        zip(nombres_perfiles, perfiles, scores),
        key=lambda x: x[2],
        reverse=True
    )

    for nombre, texto, score in resultados:
        print(f"\nPerfil: {nombre}")
        print(f"Score: {score:.4f}")
       # print("Preview:", texto[:220].replace("\n", " "), "...")
        print("-"*80)



perfiles_exp_3 = [
    # Perfil 1 – sin experiencia
    """Persona joven, mayor de edad, sin experiencia formal en ventas.
    Interés por la moda y alta disposición para aprender. Busca su primera experiencia laboral
    en tienda de ropa, con disponibilidad total de horarios.""",

    # Perfil 2 – junior 6 meses
    """Persona joven, mayor de edad, con 6 meses de experiencia como asesor(a) de ventas
    en tienda de ropa y calzado. Atención al cliente, apoyo en probadores, orden de tienda
    y cumplimiento de metas básicas de ventas.""",

    # Perfil 3 – senior 5 años
    """Persona con más de 5 años de experiencia como asesor(a) de ventas en tiendas de moda,
    manejo de indicadores, entrenamiento de personal nuevo y apoyo en inventarios.
    Busca estabilidad y crecimiento como líder de equipo."""
]

nombres_exp_3 = [
    "Sin experiencia",
    "Junior 6 meses",
    "Senior 5 años"
]

correr_experimento(
    nombre_experimento="Experimento 3 - Sensibilidad a la experiencia (junior vs senior)",
    vacante_text_list=vacante,
    perfiles=perfiles_exp_3,
    nombres_perfiles=nombres_exp_3
)



perfil_corto_relevante = """
Asesor de ventas en tienda de ropa femenina durante 1 año.
Atención al cliente, probadores, orden de tienda y metas de ventas.
"""

perfil_largo_poco_relevante = """
Profesional con amplia experiencia en distintos sectores. Ha trabajado en logística,
administración, proyectos y manejo de proveedores. Posee habilidades de comunicación,
liderazgo y adaptabilidad. Ha participado en múltiples iniciativas de mejora continua,
automatización de procesos, desarrollo de indicadores de gestión y coordinación de equipos
multidisciplinarios. Familiarizado con manejo de inventarios y recepción de mercancía, pero
sin experiencia directa en punto de venta de moda ni en asesoría de imagen. Busca nuevas
oportunidades de crecimiento profesional en empresas organizadas, con cultura fuerte y
procesos bien definidos.
"""

perfiles_exp_4 = [perfil_corto_relevante, perfil_largo_poco_relevante]
nombres_exp_4 = ["Corto pero muy relevante", "Largo pero poco relevante"]

correr_experimento(
    nombre_experimento="Experimento 4 - Ruido y longitud del texto",
    vacante_text_list=vacante,
    perfiles=perfiles_exp_4,
    nombres_perfiles=nombres_exp_4
)



RESULTADOS Experimento 3 - Sensibilidad a la experiencia (junior vs senior)

Perfil: Sin experiencia
Score: 0.4500
--------------------------------------------------------------------------------

Perfil: Senior 5 años
Score: 0.3238
--------------------------------------------------------------------------------

Perfil: Junior 6 meses
Score: 0.3117
--------------------------------------------------------------------------------

RESULTADOS Experimento 4 - Ruido y longitud del texto

Perfil: Corto pero muy relevante
Score: 0.3107
--------------------------------------------------------------------------------

Perfil: Largo pero poco relevante
Score: 0.2388
--------------------------------------------------------------------------------
